In [1]:
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.vectorstores import Milvus
from langchain.document_loaders import TextLoader
from langchain.text_splitter import CharacterTextSplitter
import os
import ipywidgets as widgets
from IPython.display import display
from dotenv import load_dotenv
from pprint import pprint

In [2]:
load_dotenv()  # take environment variables from .env.
openai_api_key = os.getenv('OPENAI_API_KEY')
milvus_api_key = os.getenv('MILVUS_API_KEY')

connection_args={
        "uri": "https://in03-5052868020ac71b.api.gcp-us-west1.zillizcloud.com",
        "user": "vaclav@pechtor.ch",
        "token": milvus_api_key,
        "secure": True
    }


In [17]:
import os
import re

def transform_source_to_url(source_value: str) -> str:
    """
    Transforms a source file path into a valid URL.
    
    Args:
        source_value: A string representing the source file path.

    Returns:
        A string representing the transformed URL.
    """
    
    # Use regular expression to find the start of the URL in the string
    match = re.search(r'([a-z]+\.[a-z]+\.[a-z]+)', source_value)
    if match:
        url_start_index = match.start()
    else:
        raise ValueError("Cannot find a URL in the source value.")
    
    # Extract the part of the string that corresponds to the URL
    url_part = source_value[url_start_index:]
    
    # Replace '__' with '/'
    url_part = url_part.replace("__", "/")
    
    # Remove the '.txt' extension
    url_part = os.path.splitext(url_part)[0]
    
    # Add the base URL to the start
    final_url = "https://" + url_part
    
    return final_url


In [4]:
embeddings = OpenAIEmbeddings()

milvus = Milvus(embedding_function=embeddings, collection_name='LuGPT', connection_args=connection_args)

In [38]:
query = "Welche Projekte macht die Dienststelle Informatik?"
print(f"Query: {query}")
docs = milvus.similarity_search(query)

print(docs)

Query: Welche Projekte macht die Dienststelle Informatik?
[Document(page_content='Sie haben Ideen und Anforderungen. Sie wollen klare Lösungen von einem kompetenten Partner. Sie verlangen die zügige Digitalisierung Ihrer Geschäftsprozesse. Die von uns entwickelten und betriebenen Basisinfrastrukturlösungen wie Netzwerke, Storage und Server stellen einen hochverfügbaren Service sicher. Mit der angebotenen Workplace-Infrastruktur arbeiten rund 6000 Nutzer flexibel und mobil. Dabei verwalten wir die zentralen Anwendungen für Kollaboration, Geschäftsfallverwaltung, Content Management und ERP. Die über 1000 Fachanwendungen mit ihren Geschäftsprozessen gehören in den Verantwortungsbereich der\xa0Departemente. Wir lösen die Integration in die Sicherheits- und Systeminfrastruktur und schützen die kantonalen Daten in der privaten und öffentlichen Cloud. Wir leisten mit Ideen und praktischen Lösungen einen substantiellen Beitrag zur Digitalisierung des Kantons. Erfolgreiche Innovationen werden i

In [39]:
from langchain.prompts import PromptTemplate
language = "German"
prompt_template = f"""You are an assistant that answers questions about the Kanton Luzern,
 based on given information. Only use the information that was provided below.
 Use the following pieces of context to answer the question at the end.
 If you don't know the answer, just say that you don't know, don't try to make up an answer.

{{context}}

Question: {{question}}
Answer in {language}:"""
PROMPT = PromptTemplate(
    template=prompt_template, input_variables=["context", "question"]
)
chain_type_kwargs = {"prompt": PROMPT}

In [40]:
from langchain.chains.qa_with_sources import load_qa_with_sources_chain
from langchain.chains import RetrievalQA
from langchain.chat_models import ChatOpenAI

if language == 'German':
    model_type = 'gpt-3.5-turbo-16k-0613'
else:
    model_type = 'gpt-4-0613'

llm = ChatOpenAI(model_name=model_type,temperature=0.0)
qa = RetrievalQA.from_chain_type(llm=llm, 
                                 retriever=milvus.as_retriever(),
                                 chain_type='stuff',
                                 return_source_documents=True,
                                 chain_type_kwargs=chain_type_kwargs)
result = qa({"query": query})

In [41]:
# Print result and source
result_value = result['result']
print(f'Query: {query}')
pprint(f'Answer: {result_value}')

source_value1 = result['source_documents'][0].metadata['source']
print(f'Source: {transform_source_to_url(source_value1)}')
source_value2 = result['source_documents'][1].metadata['source']
print(f'Source: {transform_source_to_url(source_value2)}')
source_value3 = result['source_documents'][2].metadata['source']
print(f'Source: {transform_source_to_url(source_value3)}')
source_value4 = result['source_documents'][3].metadata['source']
print(f'Source: {transform_source_to_url(source_value4)}')


Query: Welche Projekte macht die Dienststelle Informatik?
('Answer: Die Dienststelle Informatik führt zahlreiche '
 'Basisinfrastrukturprojekte durch und analysiert die Kernprozesse in den '
 'Dienststellen, um ihr Digitalisierungspotenzial zu bewerten. Sie arbeitet '
 'auch an der Erweiterung des Enterprise Service Bus, um den Anforderungen des '
 'E-Government gerecht zu werden. Weitere Projekte werden in Zusammenarbeit '
 'mit den Kunden entwickelt und umgesetzt, um Medienbrüche abzubauen und die '
 'Digitalisierung des Kantons voranzutreiben.')
Source: https://informatik.lu.ch/was_wir_bewegen
Source: https://www.lu.ch/verwaltung/FD/fd_dienststellen/informatik_lu_ch
Source: https://informatik.lu.ch/Aktuelle_Projekte/Digitaler_Kanton
Source: https://informatik.lu.ch/Organisation
